# Signal processing course 2018/2019-1 @ ELTE
# Assignment 1
## 09.17.2018

## Task 3
### Floating-point precision

![Large numbers](../img/largenumbers.jpg)

#### [Guard Digits](https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html)
One method of computing the difference between two floating-point numbers is to compute the difference exactly and then round it to the nearest floating-point number. This is very expensive if the operands differ greatly in size. Assuming $p = 3$ (see Task 5)

$$
6.02 \times 10^{23} + 23
$$

would be calculated as

$$
x = 6.02 \times 10^{23}
$$
$$
y = 0.00000000000000000000023 × 10^{23}
$$
$$
x + y = 6.02000000000000000000023 \times 10^{23}
$$

which rounds to $6.02 \times 10^{23}$. Rather than using all these digits, floating-point hardware normally operates on a fixed number of digits. Suppose that the number of digits kept is $p$, and that when the smaller operand is shifted right, digits are simply discarded (as opposed to rounding). Then 

$$
6.02 \times 10^{23} + 23
$$

becomes

$$
x = 6.02 \times 10^{23}
$$
$$
y = 0.00 \times 10^{23}
$$
$$
x + y = 6.02 \times 10^{23}
$$

The answer is exactly the same as if the summation had been computed exactly and then rounded. Take another example: $10.1 - 9.93$. This becomes

$$
x = 1.01 \times 10^{1}
$$
$$
y = 0.99 \times 10^{1}
$$
$$
x - y = 0.02 \times 10^{1}
$$

The correct answer is 0.17, so the computed difference is off by 30 ULPs (see Task 5) and is wrong in every digit!

In [ ]:
import decimal
import fractions

import numpy as np

#### Scientific notation formatter
Define a helper function that formats large numbers to regular scientific format from Python variables.

In [ ]:
def scientific(num):
    # Split the number into its components
    num_str = f'{num:.6e}'
    # Split the number into the coefficient and exponent parts
    c, p = num_str.split('e')
    # Return the formatted string
    return f'{c}*10^{int(p)}'

#### Testing native case

In [ ]:
N = 6.02214e+23
n1 = 23
n2 = 2042

print(f'Naive test 1: {scientific(N)} + {n1} = {N+n1}')
print(f'Naive test 2: {scientific(N)} + {n2} - {scientific(N)} = {N + n2 - N}')

#####  Discussion

The second equation returns the (floating-point) value of `0.0`. The reason for this is that we've reached the limits of the IEEE 64-bit floating-point type. To eliminate this limit, we need to use a data type that has a higher accuracy and covers the numberrange, which we covers in our calculations. For example, in Python 3.x, we can to use the ['decimal' library's](https://docs.python.org/3/tutorial/floatingpoint.html) Decimal(), or the 'fractions' library's Fraction() function.

### Solutions to this issue

#### 1) Decimal Library

##### Smart addition function - Decimal library

The function should look like this

```python
decimal.Decimal(str(a)) + decimal.Decimal(str(b))
```

**Explanation**: If we want to preserve accuracy, we need to pass the full values as strings, that is why we convert them forst to strings. Then we need to use the decimal.Decimal() datatype, which could add numbers with very high accuracy.

In [ ]:
def smart_add(a, b, prec=64):
    # Set precision of decimal
    decimal.getcontext().prec = prec
    # Inputs and output need to be converted to string
    return f'{decimal.Decimal(str(a)) + decimal.Decimal(str(b))}'

In [ ]:
# Test 1: Addition
smart_add(N, 0.12345678901234567)

In [ ]:
# Test 2: Subtraction
smart_add(N, -0.12345678901234567)

#### Test with examples above

In [ ]:
r1 = smart_add(N, n1)
print(f'Add. test 1: {scientific(N)} + {n1} = {r1}')

r2 = smart_add(N, n2)
print(f'Add. test 2: {scientific(N)} + {n2} = {r2}')

r3 = smart_add(smart_add(N, n2), -N)
print(f'Add. test 3: {scientific(N)} + {n2} - {scientific(N)} = {r3}')

#### Smart Multiplication - Decimal library

In [ ]:
def smart_mult(a, b, prec=64):
    # Set precision of decimal
    decimal.getcontext().prec = prec
    # Inputs and output need to be converted to string 
    return f'{decimal.Decimal(str(a)) * decimal.Decimal(str(b))}'

In [ ]:
m1 = '1.0'
m2 = '0.00000200000040000005004'

print(f'Naive test: {m1} * {m2} = {float(m1) * float(m2)}')
print(f'Mult. test: {m1} * {m2} = {smart_mult(m1, m2)}')

#### 2) Kahan Summation - variations

##### Smart addition function - [Kahan Summation](https://en.wikipedia.org/wiki/Kahan_summation_algorithm)

Creating a smart addition function, which does not limited by guard digits, could be useful, when large and very small numbers need to be added, or substracted precisely. To achieve this, we will use a Kahan Summation algorithm.

In [ ]:
def kahan_sum(X):
    '''
    Implements the naive Kahan summation algorithm.
    
    Parameters:
    X : list or array-like
        Input array of numbers to be added or subtracted
    '''
    # The final Kahan sum; starts with zero
    summ = 0
    # A running compensation for lost low-order bits
    comp = 0

    # Iterate over all numbers in the input array
    for xi in X:
        # So far, so good: Compensation is zero
        y_Kahan = xi - comp
        # Alas, `summ` is big, `y_Kahan` small, so low-order digits of
        # `y_Kahan` are lost
        temp = summ + y_Kahan
        # (`temp` - `summ`) cancels the high-order part of `y_Kahan`
        # Subtracting `y_Kahan` recovers negative (low part of `y_Kahan`)
        comp = (temp - summ) - y_Kahan
        # Algebraically, `comp` should always be zero, although
        # overly-aggressive optimizing compilers could break this in
        # various programming languages. In Python, we're safe for now.
        summ = temp
        # In the next iteration, the lost low part will be added to
        # `y_Kahan` in a fresh attempt
    return summ

In [ ]:
a_K = 120000000000000000000000000000000
b_K = 0.0000000000000000000000000000154

X_K = np.array([
    decimal.Decimal(str(a_K)), decimal.Decimal(str(b_K))
])

print(f'Kahan sum: {kahan_sum(X_K)}')

#### Smart addition function - Enhanched Kahan Summation: [Neumaier Summation](https://en.wikipedia.org/wiki/Kahan_summation_algorithm#Further_enhancements)

Neumaier introduced a slight modification of Kahan's algorithm that also covers the case when the next term to be added is larger in absolute value than the running sum, effectively swapping the role of what is large and what is small.

In [ ]:
def neumaier_sum(X):
    '''
    Implements the enhanced Kahan summation algorithm, also called as
    Neumaier Summation.
    
    Parameters:
    X : list or array-like
        Input array of numbers to be added or subtracted
    '''
    # The final Neumaier sum; starts with the first number in the input array
    summ = X[0]
    # A running compensation for lost low-order bits
    comp = 0

    # Iterate over all numbers in the input array
    for xi in X[1:]:
        # Try adding the next element to our sum so far
        temp = summ + xi
        if(np.abs(summ) >= np.abs(xi)):
            # If sum is bigger, low-order digits of `xi` are lost
            comp += (summ - temp) + xi
        else:
            # Else, low-order digits of sum are lost
            comp += (xi - temp) + summ
        summ = temp
        # Correction only applied once in the very end
    return summ + comp              

In [ ]:
a_N = 120000000000000000000000000000000
b_N = 0.0000000000000000000000000000154

X_N = np.array([
    decimal.Decimal(str(a_N)), decimal.Decimal(str(b_N))
])

print(f'Neumaier sum: {neumaier_sum(X_N)}')

### Difference between the Kahman- and the Neumaier Summation

In [ ]:
a_test = 1.0
b_test = 1e+100
c_test = 1.0
d_test = -1e+100

X_test = np.array([
    decimal.Decimal(str(v_test)) for v_test in [a_test, b_test, c_test, d_test]
])

**Correct answer should be trivially 2.0**

In [ ]:
print(f'Kahan sum: {kahan_sum(X_test)}')
print(f'Neumaier sum: {neumaier_sum(X_test)}')